# Infos

source
- https://www.geeksforgeeks.org/python-find-current-weather-of-any-city-using-openweathermap-api/
- https://openweathermap.org/

# import packages

In [47]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import csv
%matplotlib inline
import requests, json 
import os

# import API Key

In [2]:
with open ("api_key/api_key.txt", "r") as myfile:
    data=myfile.readlines()
api_key = data[0]

# Get Data via API Call

In [3]:
# Enter your API key here 
def weather_data_call(api_key):
    
    # base_url variable to store url 
    base_url =  "http://api.openweathermap.org/data/2.5/weather?"

    # Give city name 
    city_name = 'Berlin,de'
    complete_url = base_url + "appid=" + api_key + "&q=" + city_name 

    # get method of requests module 
    # return response object 
    response = requests.get(complete_url) 

    # json method of response object  
    data_json = response.json()
    return response

In [4]:
#the call
x=weather_data_call(api_key)
#x.content
#x.json()

# Export as CSV

In [5]:
y=x.json()

In [6]:
def format_values(y):
    vals=[str(dt.datetime.fromtimestamp(y['dt'])), # 
          round((y['main']['temp']-273.15),2),
          round((y['main']['humidity']),2),
          round((y['main']['pressure']),2),
          round((y['wind']['speed']),2),
          y['wind']['deg'],
          y['clouds']['all'],
          y['weather'][0]['main'],
          y['weather'][0]['description'],
          y['weather'][0]['icon'],
          str(dt.datetime.fromtimestamp(y['sys']['sunrise'])).split(' ')[1],
          str(dt.datetime.fromtimestamp(y['sys']['sunset'])).split(' ')[1]         
         ]
    
    return vals

## file name

In [61]:
export_file_name='current/current_weater_berlin.csv'

## header

In [62]:
header=["datetime","temperature","humidity","pressure","windspeed","winddir","cloudcoverage","weather","description","weathericon","sunrisetime","sunsettime"]

## data point

In [65]:
data_point=dict(zip(header, format_values(y)))
data_point

{'datetime': '2020-03-13 09:49:43',
 'temperature': 8.04,
 'humidity': 61,
 'pressure': 1008,
 'windspeed': 8.7,
 'winddir': 230,
 'cloudcoverage': 20,
 'weather': 'Clouds',
 'description': 'few clouds',
 'weathericon': '02d',
 'sunrisetime': '06:24:46',
 'sunsettime': '18:06:53'}

## append to file

In [64]:
#append to csv file
with open(export_file_name, 'a', newline='') as f:
    header=["datetime","temperature","humidity","pressure","windspeed","winddir","cloudcoverage","weather","description","weathericon","sunrisetime","sunsettime"]
    writer = csv.DictWriter(f,delimiter=',',fieldnames=header)
    if os.path.getsize(export_file_name)==0:
           writer.writeheader()          
    writer.writerow(data_point)

# create Json database